In [96]:
import json
import os

import ffmpegio
import pandas as pd
import plotly.express as px
import torch
from torchvision.transforms import Resize, Normalize, Compose
from transformers import AutoFeatureExtractor, SwinForImageClassification


def convert_number(num: str) -> int:
    """
    Убирает суффиксы К и М из чисел

    :param num: число, которое нужно преобразовать;
    :return: преобразованное число
    """
    if 'K' in num:
        return int(float(num[:-1]) * 1000)
    elif 'M' in num:
        return int(float(num[:-1]) * 1000000)
    else:
        return int(num)


renderer = None  # Как будет сохраняться график в ноутбуке

In [97]:
distr = []

for json_file in os.listdir('../../data/jsons'):
    if json_file.endswith('.json'):
        posts_data = json.load(open(f"{'../../data/jsons'}/{json_file}", 'r'))
        for post in posts_data:
            if 'likes' in post and 'views' in post:
                likes = convert_number(post['likes'])
                views = convert_number(post['views'])
                distr.append(likes / views)

fig_distr_lv = px.histogram(pd.Series(distr, name='лайки к просмотрам'),
                            title='<b>Распределение лайков к просмотрам видео</b>')
fig_distr_lv.show(renderer=renderer)

In [98]:
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
model = SwinForImageClassification.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

In [110]:
pic = ffmpegio.image.read(
    '/Users/victorbarbarich/PycharmProjects/nueramic/vktrbr-video-tiktok/data/previews/6870202569805876486.jpg')

pic: torch.Tensor = torch.tensor(pic).float()

In [107]:
transform = Compose([
    Resize((224, 224)),
    Normalize(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375])
])

In [ ]:
model(transform(pic.unsqueeze(dim=-1)))

In [88]:
px.histogram(model(torch.rand(1, 3, 224, 224) ** 2)[0].flatten().detach().numpy())